In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [ ]:
model = load_model('E:/Datasets/emojinator.h5')

In [ ]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = 'E:/Datasets/Image_data/hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()


0
1
2
3
4
5
6
7
8
9
10
11
12
1 0.99994314
1 0.98185366
1 0.9999746
1 0.99797255
1 0.99999976
1 0.9999621
1 0.99999964
11 0.97392875
3 0.9981013
1 0.9788127
1 0.9850855
1 0.9999112
8 0.9980404
3 0.99654764
1 0.3880064
1 0.9991722
1 0.54245335
11 0.55062467
11 0.9571997
11 0.9803997
1 0.99895465
9 0.987812
5 0.9719214
5 0.9150804
5 0.89361775
5 0.96618396
5 0.89343196
5 0.9389224
5 0.82976294
7 0.74341685
4 0.94872206
6 0.5967365
4 0.920432
4 0.44213635
6 0.98324585
4 0.82197934
7 0.55774385
2 0.63021743
7 0.9953258
7 0.954829
7 0.970392
5 0.60870486
5 0.599899
7 0.63084775
5 0.9725457
7 0.9898491
5 0.86158353
7 0.99183476
7 0.9737255
7 0.89570796
7 0.9998628
7 0.88319963
7 0.9998661
7 0.996718
7 0.77907044
7 0.9700038
7 0.9990559
7 0.9995171
7 0.9993647
7 0.99913245
7 0.999398
7 0.9982895
7 0.99831176
7 0.99690056
7 0.8597739
7 0.9870437
7 0.98083067
7 0.98083067
7 0.99864894
7 0.99789137
7 0.9947633
7 0.9883725
7 0.97845215
3 0.9956934
4 0.82745636
4 0.85300076
6 0.49996227
6 0.8968154

7 0.9957278
7 0.98725235
7 0.98395145
7 0.9174582
7 0.9865818
7 0.9910042
7 0.97792566
7 0.99020433
7 0.9928598
7 0.9974942
7 0.99829346
7 0.99974436
7 0.9997719
7 0.99993384
7 0.9995839
7 0.9999218
7 0.99994016
7 0.99997604
7 0.9999151
7 0.9999367
7 0.9999851
7 0.99997604
7 0.999987
7 0.99994624
7 0.999992
7 0.99999416
7 0.9990646
7 0.99694127
7 0.98254436
5 0.6634594
7 0.55742854
7 0.65558
5 0.5737441
7 0.7979412
5 0.7048159
7 0.56974584
5 0.5207342
7 0.8443242
5 0.68888336
7 0.5624591
5 0.61243504
7 0.80430204
7 0.56571716
7 0.84621716
7 0.6985726
7 0.823379
7 0.7162395
7 0.7539528
7 0.87900966
7 0.9829181
7 0.99198806
7 0.99873894
7 0.9991166
7 0.9996074
7 0.9990396
7 0.99897987
7 0.9985191
7 0.9985102
7 0.999967
7 0.9999622
7 0.99989915
7 0.99996054
7 0.9999734
7 0.99998474
7 0.99998474
7 0.9999654
7 0.9999708
7 0.99998164
7 0.99998116
7 0.9999807
7 0.999979
7 0.99996805
7 0.9999478
7 0.9999715
7 0.9999906
7 0.99998665
7 0.9999863
7 0.99998045
7 0.99998844
7 0.99995756
7 0.9999747

11 0.99999905
11 1.0
11 0.9999987
11 0.99999714
11 0.9999993
11 0.9999962
11 0.9999999
11 0.9999982
11 0.9999988
11 0.99999154
11 0.99999833
11 0.9999988
11 0.9999994
11 0.99999774
11 0.999997
11 0.9999856
11 0.99999976
11 0.99999785
11 0.9999994
11 0.99999905
11 0.9999968
11 0.9999976
11 0.999997
11 0.9999993
11 0.99999845
11 0.99999976
11 0.99999714
11 1.0
11 0.99999976
11 0.9999993
11 1.0
11 0.9999995
11 1.0
11 1.0
11 1.0
11 0.99999785
11 1.0
11 0.9999982
11 1.0
11 1.0
11 0.9999995
11 0.9999999
11 0.9999995
11 0.99999905
11 0.9999993
11 1.0
11 1.0
11 1.0
11 1.0
11 0.9999994
11 0.98068994
11 0.85115945
1 0.9588312
1 0.9241201
8 0.48759308
8 0.85774875
8 0.8766161
8 0.9348551
8 0.93563825
8 0.73693424
2 0.69615155
2 0.8045775
2 0.5619227
2 0.9874756
2 0.9330167
2 0.9966804
2 0.9947265
2 0.99519324
2 0.9947944
2 0.9894077
5 0.900875
5 0.9969367
5 0.9999896
5 0.999956
5 0.9998889
5 0.99999416
5 0.9999924
5 0.99995995
5 0.99997807
5 0.99324554
5 0.45329356
5 0.99980444
5 0.99047685
5 0.9

6 0.8883481
7 0.43870035
6 0.96443653
11 0.9978071
11 0.99916005
11 0.9996624
11 0.9540742
6 0.98178446
11 0.99953556
11 0.99994445
11 0.9989574
11 0.999877
11 0.9999076
11 0.99995875
11 0.9998871
11 0.99907815
11 0.99854845
11 0.99900836
11 0.9994093
11 0.98102415
11 0.9998628
11 0.99988914
11 0.9997111
11 0.99993384
11 0.99989164
11 0.99950707
11 0.98584753
11 0.99685293
11 0.9992673
11 0.9983364
11 0.9988827
11 0.99969435
11 0.9962639
11 0.9995142
11 0.9993819
11 0.99974805
11 0.99993074
11 0.998899
11 0.9995726
11 0.9998658
11 0.998495
11 0.9996563
6 0.9591644
11 0.9999757
11 0.99990654
11 0.99628854
11 0.99964666
11 0.99434835
11 0.99858814
11 0.9997986
11 0.88189393
11 0.9903218
11 0.9958205
6 0.99958974
4 0.5184677
6 0.97125816
6 0.46993113
9 0.46820417
3 0.8879897
3 0.39505413
9 0.9958248
9 0.99948597
6 0.5928458
9 0.84807634
6 0.9005466
4 0.9901126
5 0.9902739
5 0.95592654
5 0.7116525
5 0.87621325
5 0.9202549
7 0.583208
5 0.6317831
5 0.9929097
5 0.99956375
5 0.927503
5 0.99745

5 0.9945967
5 0.99796593
5 0.9981482
5 0.99492514
5 0.9890633
5 0.9969842
5 0.99465203
5 0.9461591
5 0.9897899
5 0.79917103
7 0.5338985
7 0.8682174
5 0.72873354
11 0.9337347
8 0.99998116
8 0.99999714
8 0.9999926
8 0.9999975
8 0.9999248
8 0.99997234
8 0.99998283
8 0.99998283
8 0.99999475
8 0.99993825
8 0.9999908
8 0.9999112
8 0.99985754
11 0.99878055
8 0.9999087
8 0.9999119
11 0.9917619
11 0.99398017
11 0.99398017
8 0.99987996
8 0.9999198
8 0.99999547
8 0.9999585
8 0.99969995
8 0.9999467
8 0.9998579
11 0.99482214
8 0.99986196
8 0.9978916
8 0.99996424
8 0.99891233
8 0.99992275
8 0.9999825
8 0.9999603
8 0.99989784
8 0.9778581
8 0.92790896
11 0.99814916
11 0.99800986
4 0.8856375
7 0.9551413
11 0.999642
11 0.9988482
1 0.9972199
5 0.8560892
5 0.9944635
1 0.97583073
9 0.9174718
5 0.9960536
5 0.99066913
5 0.9067469
5 0.9771739
10 0.9999311
9 0.99834883
1 0.9999974
3 0.4975105
6 0.98667395
1 0.99998426
10 0.6861458
11 0.32292768
2 0.7850794
8 0.7751192
9 0.9867953
6 0.5211156
10 0.9949116
9 0.9

...

3 0.53533226
4 0.76632744
3 0.47061363
3 0.74946004
3 0.70501107
4 0.51346374
7 0.8202089
7 0.8679142
7 0.79163516
7 0.99772424
7 0.9995018
7 0.9973086
7 0.9999496
7 0.99999523
7 0.9999976
7 0.9999964
7 0.99999523
7 0.99999726
7 0.99999774
7 0.9999994
7 0.99999905
7 0.9999994
7 0.99999964
7 0.9999981
7 0.9999995
7 0.9999994
7 0.9999994
7 0.99999964
7 0.9999995
7 0.9999995
7 0.99999726
7 0.9999995
7 0.9999995
7 0.9999988
7 0.99999774
7 0.9999962
7 0.9999981
7 0.9999964
7 0.99999845
7 0.99999344
7 0.9999969
7 0.9999976
7 0.99999976
7 0.99999964
7 0.99999964
7 0.9999989
7 0.99999917
7 0.99999964
7 0.99999785
7 0.9999974
7 0.99999166
7 0.9999943
7 0.9999964
7 0.9999962
7 0.9999938
7 0.99999225
7 0.99997985
7 0.99999607
7 0.9999765
7 0.99998844
7 0.9999746
7 0.9999763
7 0.999979
7 0.99996495
7 0.99993944
7 0.9992949
7 0.99882585
7 0.999926
7 0.9999187
7 0.99991226
7 0.9999001
7 0.9999622
7 0.9999583
7 0.9997004
7 0.99990976
7 0.99994636
7 0.99995434
7 0.99994636
7 0.9999459
7 0.99995315
7 0

11 0.79098666
11 0.9018614
11 0.5942749
8 0.862657
8 0.99876535
8 0.97608143
8 0.97608143
8 0.8796822
8 0.9333372
8 0.36146396
11 0.85482675
9 0.9936783
9 0.9999262
9 0.9996445
9 0.9544101
9 0.99925715
9 0.9980354
9 0.9988701
9 0.97848094
9 0.9991565
9 0.998379
9 0.9999778
9 0.99998987
9 0.9998191
9 0.9864494
9 0.9990771
9 0.99983895
9 0.9999391
9 0.99918646
9 0.99982625
9 0.98576385
9 0.9999745
9 0.99993443
9 0.99990225
9 0.92802674
9 0.96129644
9 0.99998164
9 0.95999616
9 0.99978155
9 0.99850595
9 0.999592
9 0.9997737
9 0.9972698
9 0.9999864
9 0.99989367
9 0.99985504
9 0.9999676
9 0.9998155
9 0.99839264
9 0.9883604
9 0.9995653
9 0.99973327
9 0.9997124
9 0.9997652
9 0.98207194
8 0.8569222
8 0.9894098
8 0.6357441
5 0.7073641
1 0.7627822
5 0.7839709
5 0.65624857
5 0.79427457
1 0.70580345
5 0.6457525
5 0.8593384
5 0.96190673
5 0.96190673
5 0.98013043
5 0.9548141
5 0.9786283
5 0.956031
11 0.97282594
11 0.99106276
11 0.9951454
11 0.99945205
11 0.9974879
11 0.97318244
11 0.79396516
11 0.952

6 0.99150133
6 0.9607388
6 0.9992993
6 0.95796967
6 0.9593436
6 0.94845176
6 0.98823476
6 0.9940065
6 0.9319264
6 0.6551004
6 0.51340264
10 0.9967601
10 0.9716333
10 0.9468461
10 0.8027355
10 0.8417521
6 0.84936154
10 0.77666837
7 0.51726794
7 0.86167264
2 0.8053951
1 0.6808093
1 0.98932195
1 0.99999344
1 0.9999969
1 0.9999932
1 0.99995637
1 0.99988985
1 0.99970645
1 0.9895147
1 0.99334395
1 0.99531174
1 0.9965313
1 0.98582816
1 0.9943303
1 0.98578554
1 0.995413
1 0.9989926
1 0.9990631
1 0.98221844
1 0.99511695
1 0.92964685
1 0.9947023
1 0.9979335
1 0.9991755
1 0.9975224
1 0.9999181
1 0.9990809
1 0.99969625
1 0.9999038
1 0.9997904
1 0.9997851
1 0.9997179
1 0.9998062
1 0.99942434
1 0.99967694
1 0.99971646
1 0.99992704
1 0.95472914
1 0.7457173
6 0.48000348
8 0.60107857
8 0.86165786
8 0.77750957
2 0.96330506
2 0.99986804
2 0.9998053
2 0.9997209
2 0.9997683
8 0.98600185
1 0.96549594
1 0.9993693
1 0.99929833
1 0.98587453
1 0.99892277
1 0.99900085
1 0.9882427
1 0.97687566
1 0.9579919
1 0.997

6 0.99883443
6 0.9998765
6 0.9997602
6 0.99915814
6 0.9996656
6 0.9998049
6 0.9999809
6 0.99984646
6 0.99998915
6 0.9997924
6 0.99997246
6 0.9996612
6 0.9999702
6 0.9930327
6 0.99975854
6 0.99903154
6 0.999782
6 0.999972
6 0.9999608
6 0.99990547
6 0.99841785
6 0.9999218
6 0.9998951
6 0.9997999
6 0.9997625
6 0.99991775
6 0.9999374
6 0.99995136
6 0.9996401
6 0.9998529
6 0.9997627
6 0.9974347
6 0.9991221
6 0.9945281
6 0.99864024
6 0.99903643
6 0.9993292
9 0.9689233
9 0.9332183
6 0.68231887
9 0.65509945
9 0.7159679
9 0.9946826
9 0.7315795
9 0.96757877
9 0.98877966
9 0.9991406
9 0.9989048
9 0.99855286
9 0.9935202
9 0.897187
9 0.74635893
9 0.88147473
6 0.84247917
9 0.74024135
3 0.69187844
8 0.9544329
11 0.9894049
11 0.7808631
10 0.99981314
5 0.9356863
5 0.66581637
7 0.98685
7 0.99719006
7 0.99531615
7 0.96285087
7 0.78093874
7 0.7474954
7 0.7240558
7 0.6164714
7 0.82535857
7 0.56949383
6 0.6157645
6 0.63975
7 0.64694405
7 0.7044664
7 0.5765038
5 0.4840993
5 0.85572594
5 0.6480843
5 0.8550215

2 0.51302695
3 0.40643603
2 0.80810577
2 0.9117654
2 0.975662
2 0.8908671
2 0.85318035
2 0.85318035
2 0.94011396
2 0.99344975
2 0.8568861
2 0.97442484
11 0.8284279
9 0.98541427
1 0.5240365
5 0.4202871
5 0.6503693
10 0.99996436
1 0.8769332
10 0.99990904
10 0.9994905
10 0.9933478
1 0.42341718
10 0.9905929
10 0.99754
10 0.9990734
10 0.9990734
10 0.99923205
1 0.87674063
11 0.58012724
1 0.78099823
1 0.71373236
1 0.9965753
11 0.525921
3 0.5050644
3 0.6625591
3 0.9658433
3 0.96095186
3 0.8612587
1 0.6588986
9 0.6314198
1 0.9425288
1 0.99906844
3 0.7681165
2 0.30462667
3 0.460138
3 0.525474
3 0.8954777
2 0.78654736
2 0.51341546
2 0.59119546
1 0.6197787
1 0.67044264
9 0.4010358
9 0.4408967
1 0.42077583
1 0.60464597
1 0.97849214
1 0.93875307
1 0.9168558
2 0.8830426
1 0.94153255
1 0.9999833
1 0.9828469
1 0.9764825
1 0.9313006
1 0.9997923
9 0.8618523
1 0.9998883
9 0.8572212
9 0.9560466
1 0.98540443
1 0.89669245
1 0.76271915
1 0.99983287
1 0.5732278
1 0.9995359
1 0.9984452
1 0.9986897
1 0.99646974
